In [1]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np
def draw_korean_text(frame, text, position, font_path, font_size, color):
    font = ImageFont.truetype(font_path, font_size)
    img_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return np.array(img_pil)

font_path = "models/korean_font/NanumGothic.ttf"



In [2]:
import cv2
import mediapipe as mp
import numpy as np
from keras.models import load_model

emotions_korean = ["분노", "슬픔",  "당황", "기쁨", "중립"]
emotions = ["angry", "sad", "embarrassed", "happy", "neutral"]


model_path = "v2_final.hdf5"
# model_path = "effb0_final.hdf5"
model = load_model(model_path)
input_size = (48,48)
is_grey = True

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, ic = frame.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

            # 얼굴 부분 자르기
            face = frame[y:y+h, x:x+w]
            try:
                face = cv2.resize(face, input_size) 
                if is_grey: #흑백
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY) 
                face = face / 255.0 # 정규화
                face = np.expand_dims(face, axis=0)
                face = np.expand_dims(face, axis=-1)

                # 감정 예측
                prediction = model.predict(face,verbose = 0)
                emotion = emotions_korean[np.argmax(prediction)]

                # 텍스트로 감정 표시
                # cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                frame = draw_korean_text(frame, emotion, (x, y - 30), font_path, 20, (36,255,12))

            except Exception as e:
                print(e)
                continue

            # 얼굴에 BBOX 그리기
            # mp_drawing.draw_detection(frame, detection)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), 2)


    # 결과 보여주기
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
